In [1]:
%pip install g4f

Note: you may need to restart the kernel to use updated packages.


In [4]:
import g4f
import json
from openai import OpenAI

In [10]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."

In [10]:
def prompt_desc(desc):
    prompt = f'''
    Pease extract the following information from the given text and return it as a JSON object:

    name
    major
    school
    grades
    club
    
    This is the body of text to extract the information from:
    {desc}
    '''
    return prompt

In [11]:
desc_1 = prompt_desc(student_1_description)
desc_1

NameError: name 'student_1_description' is not defined

In [20]:
response = g4f.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role":"user","content":desc_1}
    ]
)
response

'{\n  "name": "David Nguyen",\n  "major": "computer science",\n  "school": "Stanford University",\n  "grades": "3.8 GPA",\n  "club": "Robotics Club"\n}'

In [21]:
json_response = json.loads(response)
json_response

{'name': 'David Nguyen',
 'major': 'computer science',
 'school': 'Stanford University',
 'grades': '3.8 GPA',
 'club': 'Robotics Club'}

In [22]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [23]:
desc_2 = prompt_desc(student_2_description)
desc_2

"\n    Pease extract the following information from the given text and return it as a JSON object:\n\n    name\n    major\n    school\n    grades\n    club\n    \n    This is the body of text to extract the information from:\n    Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating.\n    "

In [24]:
response_2 = g4f.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role':'user','content':desc_2}]
)
response_2

'{\n  "name": "Ravi Patel",\n  "major": "computer science",\n  "school": "University of Michigan",\n  "grades": "3.7 GPA",\n  "club": [\n    "Chess Club",\n    "South Asian Student Association"\n  ]\n}'

In [25]:
json_response_2 = json.loads(response_2)
json_response_2

{'name': 'Ravi Patel',
 'major': 'computer science',
 'school': 'University of Michigan',
 'grades': '3.7 GPA',
 'club': ['Chess Club', 'South Asian Student Association']}

Function Calling

In [27]:
student_custom_functions = [
    {
        'name':'extract_student_info',
        'decription':'Get the student information from the body of the input text',
        'parameters':{
            'type':'object',
            'properties':{
                'name':{
                    'type':'string',
                    'description':'Name of the person'
                },
                'major':{
                    'type':'string',
                    'decscription':'Major subject'
                },
                'school':{
                    'type':'string',
                    'description':"Name of University"
                },
                'grades':{
                    'type':"integer",
                    'description':"GPA of the student"
                },
                'club':{
                    'type':'string',
                    'description':'School club for extracurricular activities'
                }
            }
        }
    }
]

In [28]:
student_descriptions = [student_1_description,student_2_description]

In [5]:
from dotenv import load_dotenv,find_dotenv
_:bool = load_dotenv(find_dotenv())
client: OpenAI = OpenAI()

In [6]:
from openai.types.chat.chat_completion import ChatCompletion

In [7]:
def chat_completion(prompt:str)->str:
    response: ChatCompletion = client.chat.completions.create(
        model='gpt-3.5-turbo-1106',
        messages=[{"role":"user",'content':prompt}]
    )
    return response.choices[0].message.content

In [8]:
chat_completion("What is 1+1")

'1 + 1 equals 2.'

In [12]:
student_1 = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."
student_2 = "Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."

In [13]:
s1_desc = prompt_desc(student_1)
s2_desc = prompt_desc(student_2)

In [27]:
student_custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [28]:
student_descriptions = [s1_desc,s2_desc]
import json

In [29]:
for sample in student_descriptions:
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-1106',
        messages=[{'role':'user','content':sample}],
        functions=student_custom_functions,
        function_call='auto'
    )
    print(response.choices[0].message)

KeyboardInterrupt: 